# Instrument Recognition Based on Deep CNN

## Import Librarys

In [ ]:
import os
import numpy as np
import random
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import librosa
import matplotlib.pyplot as plt
import seaborn as sns

## Audio Preprocessing

In [24]:
# Preprocess Audio Function
def preprocess_audio(file_path,
                     target_sr=22050,
                     n_fft=1024,
                     hop_length=512,
                     n_mels=128,
                     max_length=None):
    """
    Preprocess an audio file for instrument recognition.
    Returns:
        np.ndarray: Log-mel spectrogram of shape (n_mels, time_steps)
    """
    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=target_sr, mono=True)

        # Normalize the audio signal
        if np.max(np.abs(y)) > 0:
            y = y / np.max(np.abs(y))

        # Compute Mel-Spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=y,
                                                         sr=target_sr,
                                                         n_fft=n_fft,
                                                         hop_length=hop_length,
                                                         n_mels=n_mels,
                                                         fmax=target_sr / 2)

        # Apply logarithm
        log_mel_spectrogram = np.log10(mel_spectrogram + 1e-9)  # Add epsilon to avoid log(0)

        return log_mel_spectrogram
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def precompute_mel_spectrograms(file_paths, save_dir):
    """
    Precomputes mel-spectrograms for a list of audio files and saves them to disk.
    """
    os.makedirs(save_dir, exist_ok=True)
    for file_path in tqdm(file_paths, desc="Precomputing Mel-Spectrograms"):
        relative_path = os.path.relpath(file_path, start=os.path.commonpath(file_paths))
        save_path = os.path.join(save_dir, relative_path) + '.npy'
        if not os.path.exists(save_path):
            mel_spec = preprocess_audio(file_path)
            if mel_spec is not None:
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                np.save(save_path, mel_spec)
            else:
                print(f"Failed to process {file_path}")

### Visualization of mel spectrogram

In [ ]:
# Example usage:

# Path to a sample audio file
sample_file = './../../../IRMAS/IRMAS-TrainingData/flu/008__[flu][nod][cla]0393__1.wav'  # Replace with your actual file path

# Preprocess the audio
log_mel_spec = preprocess_audio(sample_file)

print(f"Log-Mel Spectrogram shape: {log_mel_spec.shape}")

# Visualize the spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(log_mel_spec,
                         sr=22050,
                         hop_length=512,
                         x_axis='time',
                         y_axis='mel',
                         cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('Log-Mel Spectrogram')
plt.tight_layout()
plt.show()

## Training and Testing Data Process
### Loading Dataset

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import random

# Set seed for reproducibility
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # For deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Function to clean and process label strings
def clean_labels(label_text):
    """
    Cleans and splits raw label text from .txt files.
    """
    labels = [label.strip() for label in label_text.split() if label.strip()]
    # Remove any numeric-only labels or unexpected characters
    labels = [label for label in labels if not label.isdigit()]
    return labels



class IRMASDataset(Dataset):
    def __init__(self, file_paths, labels, label_encoder, mel_specs_dir, transform=None):
        """
        Custom PyTorch Dataset for IRMAS data with precomputed mel-spectrograms.
        Args:
            file_paths (list): List of audio file paths.
            labels (list): List of labels (instrument names) for each file.
            label_encoder (LabelEncoder): Fitted LabelEncoder.
            mel_specs_dir (str): Directory where precomputed mel-spectrograms are stored.
            transform (callable): Transform to apply to mel-spectrogram.
        """
        self.file_paths = file_paths
        self.labels = labels  # List of instrument labels (strings)
        self.label_encoder = label_encoder
        self.mel_specs_dir = mel_specs_dir
        self.transform = transform

        # Compute mel-spectrogram paths
        self.mel_spec_paths = [self.get_mel_spec_path(fp) for fp in self.file_paths]

    def get_mel_spec_path(self, audio_path):
        # Get the relative path from the base data directory
        relative_path = os.path.relpath(audio_path, start=os.path.commonpath(self.file_paths))
        # Construct the path to the mel-spectrogram
        mel_spec_path = os.path.join(self.mel_specs_dir, relative_path) + '.npy'
        return mel_spec_path

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        mel_spec_path = self.mel_spec_paths[idx]
        label = self.labels[idx]  # Single instrument label (string)

        # Load precomputed mel-spectrogram
        if os.path.exists(mel_spec_path):
            mel_spec = np.load(mel_spec_path)
        else:
            # Handle missing mel-spectrogram files
            print(f"Mel-spectrogram not found at {mel_spec_path}. Using zeros.")
            mel_spec = np.zeros((128, 128))  # Adjust shape if necessary

        mel_spec = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0)  # Shape: [1, n_mels, time_frames]

        # Apply any transformations
        if self.transform:
            mel_spec = self.transform(mel_spec)

        # Encode label as integer
        label_index = self.label_encoder.transform([label])[0]
        target = torch.tensor(label_index, dtype=torch.long)

        return mel_spec, target



def get_metadata(train_dir, test_dir_part1, test_dir_part2):
    """
    Collects file paths and labels from training and testing directories.
    Returns:
        train_file_paths (list): List of training audio file paths.
        train_labels (list): List of labels for training data.
        test_file_paths (list): List of testing audio file paths.
        test_labels (list): List of labels for testing data.
    """
    train_file_paths = []
    train_labels = []
    test_file_paths = []
    test_labels = []

    # Process Training Data
    for subdir in os.listdir(train_dir):
        subfolder_path = os.path.join(train_dir, subdir)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                if file.endswith(".wav"):
                    file_path = os.path.join(subfolder_path, file)
                    train_file_paths.append(file_path)
                    # Use the subdirectory name as the predominant instrument label
                    train_labels.append(subdir)

    # Process Testing Data Part 1 and Part 2
    for test_dir in [test_dir_part1, test_dir_part2]:
        if not os.path.exists(test_dir):
            continue
        for file in os.listdir(test_dir):
            if file.endswith(".wav"):
                file_path = os.path.join(test_dir, file)
                txt_file_path = os.path.splitext(file_path)[0] + ".txt"

                # Read the labels from the .txt file
                if os.path.exists(txt_file_path):
                    with open(txt_file_path, "r") as txt_file:
                        raw_labels = txt_file.read()
                        labels = clean_labels(raw_labels)
                        if labels:
                            # Select the predominant instrument (first label)
                            predominant_instrument = labels[0]
                            test_file_paths.append(file_path)
                            test_labels.append(predominant_instrument)
                        else:
                            # Exclude samples with no labels
                            continue
                else:
                    # Exclude samples with missing label files
                    continue

    return train_file_paths, train_labels, test_file_paths, test_labels

# Function to filter samples without labels
def filter_samples_without_labels(file_paths, labels):
    filtered_file_paths = []
    filtered_labels = []
    for fp, lbl in zip(file_paths, labels):
        if lbl and lbl != ['unknown']:
            filtered_file_paths.append(fp)
            filtered_labels.append(lbl)
    return filtered_file_paths, filtered_labels

# Directories
train_dir = "./../../../IRMAS/IRMAS-TrainingData"
test_dir_part1 = "./../../../IRMAS/IRMAS-TestingData-Part1/Part1"
test_dir_part2 = "./../../../IRMAS/IRMAS-TestingData-Part2/IRTestingData-Part2"

# Extract metadata
train_file_paths, train_labels, test_file_paths, test_labels = get_metadata(train_dir, test_dir_part1, test_dir_part2)

# Combine all labels from training and testing data
all_labels = train_labels + test_labels
# Remove duplicates
all_label_list = list(set(all_labels))

# Fit the LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(all_label_list)

# Precompute mel-spectrograms for all data
mel_specs_dir = './preprocessed_data'
all_file_paths = train_file_paths + test_file_paths
precompute_mel_spectrograms(all_file_paths, mel_specs_dir)

# Split training data into training and validation sets
train_files, val_files, train_labels_split, val_labels_split = train_test_split(
    train_file_paths,
    train_labels,
    test_size=0.15,
    random_state=42,
    shuffle=True,
    stratify=train_labels
)

print(f"Training samples: {len(train_files)}")
print(f"Validation samples: {len(val_files)}")

### Building Dataloader for Network

In [ ]:
# Create Dataset instances
train_dataset = IRMASDataset(
    file_paths=train_files,
    labels=train_labels_split,
    label_encoder=label_encoder,
    mel_specs_dir=mel_specs_dir
)

val_dataset = IRMASDataset(
    file_paths=val_files,
    labels=val_labels_split,
    label_encoder=label_encoder,
    mel_specs_dir=mel_specs_dir
)

# Create DataLoaders
batch_size = 128
num_workers = 4

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

### Visualization of Dataset

In [ ]:
import librosa.display
import matplotlib.pyplot as plt


# Get a sample from the training dataset
sample_mel_spec, sample_label = train_dataset[0]




# Convert tensor to numpy for visualization
sample_mel_spec_np = sample_mel_spec.squeeze(0).numpy()

# Plot the spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(sample_mel_spec_np,
                         sr=22050,
                         hop_length=512,
                         x_axis='time',
                         y_axis='mel',
                         cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title(f'Sample Log-Mel Spectrogram ')
plt.tight_layout()
plt.show()

## Deep CNN Model

In [ ]:
class InstrumentCNN(nn.Module):
    def __init__(self, num_classes):
        super(InstrumentCNN, self).__init__()

        # Convolutional Block 1
        self.conv1_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout(0.25)

        # Convolutional Block 2
        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=1)
        self.dropout2 = nn.Dropout(0.25)

        # Convolutional Block 3
        self.conv3_1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=1)
        self.dropout3 = nn.Dropout(0.25)

        # Convolutional Block 4
        self.conv4_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)

        # Global Max Pooling
        self.global_pool = nn.AdaptiveMaxPool2d((1, 1))

        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * 1 * 1, 1024)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, num_classes)

        # Initialize weights
        self._initialize_weights()

    def forward(self, x):
        # Convolutional Block 1
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        x = self.dropout1(x)

        # Convolutional Block 2
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool2(x)
        x = self.dropout2(x)

        # Convolutional Block 3
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = self.pool3(x)
        x = self.dropout3(x)

        # Convolutional Block 4
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))

        # Global Max Pooling
        x = self.global_pool(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)

        return x  # Do not apply sigmoid here

    def _initialize_weights(self):
        # Initialize weights using Xavier uniform initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

## Training

In [ ]:
# Instantiate the model
num_classes = len(label_encoder.classes_)
model = InstrumentCNN(num_classes=num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Current Training Session is Running on: ", device)
model = model.to(device)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid activation and binary cross-entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training parameters
num_epochs = 50  # You can adjust this
early_stopping_patience = 10
best_val_loss = np.inf
epochs_no_improve = 0

# Lists to store losses and metrics
train_losses = []
val_losses = []
train_f1_scores = []
val_f1_scores = []

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    all_preds = []
    all_targets = []

    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]"):
        inputs = inputs.to(device)  # Shape: [batch_size, 1, 128, frames_per_segment]
        targets = targets.to(device)  # Shape: [batch_size, num_classes]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item() * inputs.size(0)

        # Collect predictions and targets for metric computation
        preds = torch.sigmoid(outputs).detach().cpu().numpy()  # Apply sigmoid for probabilities
        targets_np = targets.detach().cpu().numpy()
        all_preds.append(preds)
        all_targets.append(targets_np)

    # Calculate average loss over the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)

    # Compute F1 Score
    all_preds = np.vstack(all_preds)
    all_targets = np.vstack(all_targets)
    # Binarize predictions with a threshold (e.g., 0.5)
    all_preds_binary = (all_preds >= 0.5).astype(int)
    train_f1 = f1_score(all_targets, all_preds_binary, average='micro', zero_division=0)
    train_f1_scores.append(train_f1)

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_all_preds = []
    val_all_targets = []

    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]"):
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, targets)

            # Accumulate loss
            val_running_loss += loss.item() * inputs.size(0)

            # Collect predictions and targets
            preds = torch.sigmoid(outputs).cpu().numpy()
            targets_np = targets.cpu().numpy()
            val_all_preds.append(preds)
            val_all_targets.append(targets_np)

    # Calculate average validation loss
    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_losses.append(val_epoch_loss)

    # Compute validation F1 Score
    val_all_preds = np.vstack(val_all_preds)
    val_all_targets = np.vstack(val_all_targets)
    val_all_preds_binary = (val_all_preds >= 0.5).astype(int)
    val_f1 = f1_score(val_all_targets, val_all_preds_binary, average='micro', zero_division=0)
    val_f1_scores.append(val_f1)

    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {epoch_loss:.4f}, Train F1: {train_f1:.4f} - "
          f"Val Loss: {val_epoch_loss:.4f}, Val F1: {val_f1:.4f}")

    # Early Stopping
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pth')
        print("Validation loss decreased. Model saved.")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print("Early stopping triggered.")
            break

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Save the final model
torch.save(model.state_dict(), 'final_model.pth')
print("Training completed. Model saved as 'final_model.pth'.")



In [ ]:
import matplotlib.pyplot as plt

# Plot Training and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot Training and Validation F1 Score
plt.figure(figsize=(10, 5))
plt.plot(train_f1_scores, label='Training F1 Score')
plt.plot(val_f1_scores, label='Validation F1 Score')
plt.title('Training and Validation F1 Score over Epochs')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.grid(True)
plt.show()

## Model Evalation

### Testset

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
import librosa
import os

class IRMASTestDataset(Dataset):
    def __init__(self, file_paths, labels, label_encoder,
                 target_sr=22050,
                 n_fft=1024,
                 hop_length=512,
                 n_mels=128,
                 window_size=3.0,
                 hop_size=None,
                 transform=None):
        """
        Dataset for test audio files with variable lengths and overlapping windows.

        Args:
            file_paths (list): List of test audio file paths.
            labels (list): List of labels (list of instruments) for each file.
            label_encoder (LabelEncoder): Fitted LabelEncoder.
            window_size (float): Window size in seconds (same as training window).
            hop_size (float): Hop size in seconds (half of window size if None).
            transform (callable): Optional transform to be applied on a sample.
        """
        self.file_paths = file_paths
        self.labels = labels
        self.label_encoder = label_encoder
        self.target_sr = target_sr
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.window_size = window_size
        self.hop_size = hop_size if hop_size is not None else window_size / 2
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        labels = self.labels[idx]

        # Load the audio file
        y, sr = librosa.load(file_path, sr=self.target_sr, mono=True)

        # Normalize the audio signal
        if np.max(np.abs(y)) > 0:
            y = y / np.max(np.abs(y))

        # Calculate the number of samples per window and hop
        window_length_samples = int(self.window_size * self.target_sr)
        hop_length_samples = int(self.hop_size * self.target_sr)

        # Compute the number of windows
        total_length = len(y)
        num_windows = int(np.ceil((total_length - window_length_samples) / hop_length_samples)) + 1

        # Segment the audio into overlapping windows
        segments = []
        for i in range(num_windows):
            start = i * hop_length_samples
            end = start + window_length_samples
            segment = y[start:end]

            # Pad segment if necessary
            if len(segment) < window_length_samples:
                pad_width = window_length_samples - len(segment)
                segment = np.pad(segment, (0, pad_width), mode='constant')

            # Compute mel-spectrogram
            mel_spec = librosa.feature.melspectrogram(
                y=segment,
                sr=self.target_sr,
                n_fft=self.n_fft,
                hop_length=self.hop_length,
                n_mels=self.n_mels,
                fmax=self.target_sr / 2
            )
            # Apply logarithm
            mel_spec = np.log(mel_spec + 1e-9)
            segments.append(mel_spec)

        # Convert list of segments to a tensor
        segments = np.array(segments)  # Shape: [num_windows, n_mels, time_frames]
        segments = torch.tensor(segments, dtype=torch.float32)

        # Add channel dimension
        segments = segments.unsqueeze(1)  # Shape: [num_windows, 1, n_mels, time_frames]

        # Apply transformations if any
        if self.transform:
            segments = self.transform(segments)

        # Convert labels to multi-hot encoding
        label_indices = self.label_encoder.transform(labels)
        target = torch.zeros(len(self.label_encoder.classes_), dtype=torch.float32)
        target[label_indices] = 1.0

        return segments, target, file_path  # Return file_path for identification

# Instantiate the test dataset
test_dataset = IRMASTestDataset(
    file_paths=test_file_paths,
    labels=test_labels,
    label_encoder=label_encoder,
    target_sr=22050,
    n_fft=1024,
    hop_length=512,
    n_mels=128,
    window_size=3.0,  # Same as training window size
    hop_size=1.5,     # Half of the window size
    transform=None
)

# Create DataLoader for the test set
test_loader = DataLoader(
    test_dataset,
    batch_size=1,  # Batch size of 1 since each item may have different number of segments
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

### Model Prediction


In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
from sklearn.metrics import (f1_score, multilabel_confusion_matrix,
                             ConfusionMatrixDisplay, precision_recall_fscore_support,
                             accuracy_score, hamming_loss)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
model.to(device)

# Instantiate the test dataset and DataLoader
test_dataset = IRMASTestDataset(
    file_paths=test_file_paths,
    labels=test_labels,
    label_encoder=label_encoder,
    target_sr=22050,
    n_fft=1024,
    hop_length=512,
    n_mels=128,
    window_size=3.0,
    hop_size=1.5,
    transform=None
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# Define thresholds
thresholds = np.linspace(0.2, 0.8, 13)
best_f1 = 0.0
best_threshold = 0.5
f1_scores = []

# Store true labels and predicted labels
all_true_labels = []
all_predicted_labels = {threshold: [] for threshold in thresholds}

# Testing loop
for segments, target, file_path in tqdm(test_loader, desc='Testing'):
    segments = segments.squeeze(0)
    target = target.to(device)
    num_segments = segments.size(0)
    segments = segments.to(device)

    # Collect outputs for all segments
    outputs = []
    with torch.no_grad():
        for i in range(num_segments):
            input_segment = segments[i].unsqueeze(0)
            logits = model(input_segment)
            sigmoid_output = torch.sigmoid(logits)
            outputs.append(sigmoid_output.cpu().numpy())

    outputs = np.vstack(outputs)

    # Aggregate predictions by averaging class-wise
    avg_outputs = np.mean(outputs, axis=0)

    # Normalize by dividing by the maximum value among classes
    if np.max(avg_outputs) > 0:
        normalized_outputs = avg_outputs / np.max(avg_outputs)
    else:
        normalized_outputs = avg_outputs

    # Store true labels
    true_labels = target.cpu().numpy()
    all_true_labels.append(true_labels)

    # Apply different thresholds and store predictions
    for threshold in thresholds:
        predicted_labels = (normalized_outputs >= threshold).astype(int)
        all_predicted_labels[threshold].append(predicted_labels)

all_true_labels = np.vstack(all_true_labels)

# Compute F1 scores for each threshold
for threshold in thresholds:
    preds = np.vstack(all_predicted_labels[threshold])
    f1 = f1_score(all_true_labels, preds, average='micro', zero_division=0)
    f1_scores.append(f1)
    print(f"Threshold: {threshold:.2f}, F1 Score: {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_preds = preds

print(f"\nBest Threshold: {best_threshold:.2f}, Best F1 Score: {best_f1:.4f}")

# Compute confusion matrices for the best threshold
confusion_matrices = multilabel_confusion_matrix(all_true_labels, best_preds)

# Display confusion matrices and metrics for each class
class_labels = label_encoder.classes_

# Compute class-wise precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(
    all_true_labels,
    best_preds,
    average=None,
    zero_division=0
)

for idx, (cm, label) in enumerate(zip(confusion_matrices, class_labels)):
    print(f"Class: {label}")
    print(f"  Precision: {precision[idx]:.4f}")
    print(f"  Recall:    {recall[idx]:.4f}")
    print(f"  F1 Score:  {f1[idx]:.4f}")
    print(f"  Confusion Matrix:\n{cm}\n")

    # Optionally display confusion matrix as a plot
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f'Not {label}', label])
    disp.plot(values_format='d')
    plt.title(f'Confusion Matrix for Class: {label}')
    plt.show()

# Calculate Exact Match Accuracy (Subset Accuracy)
exact_match_accuracy = accuracy_score(all_true_labels, best_preds)
print(f"Exact Match Accuracy (Subset Accuracy): {exact_match_accuracy:.4f}")

# Calculate Hamming Loss and Hamming Accuracy
hamming_loss_value = hamming_loss(all_true_labels, best_preds)
hamming_accuracy = 1 - hamming_loss_value
print(f"Hamming Accuracy: {hamming_accuracy:.4f}")

# Plot F1 Score vs Threshold
plt.figure(figsize=(8, 6))
plt.plot(thresholds, f1_scores, marker='o')
plt.title('F1 Score vs Threshold')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.grid(True)
plt.show()

# Compute the combined confusion matrix
num_classes = all_true_labels.shape[1]
combined_confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

for true_labels, predicted_labels in zip(all_true_labels, best_preds):
    true_indices = np.where(true_labels == 1)[0]
    pred_indices = np.where(predicted_labels == 1)[0]
    for i in true_indices:
        for j in pred_indices:
            combined_confusion_matrix[i][j] += 1

# Display the combined confusion matrix
confusion_df = pd.DataFrame(combined_confusion_matrix, index=class_labels, columns=class_labels)

plt.figure(figsize=(12, 10))
sns.heatmap(confusion_df, annot=True, fmt='d', cmap='Blues')
plt.title('Combined Confusion Matrix for All Instruments')
plt.xlabel('Predicted Instruments')
plt.ylabel('True Instruments')
plt.show()